<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/BTC_WFO_POC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner -q
!pip install ccxt -q
!pip install ta -q

In [ ]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/gdrive')
print("Drive mounted successfully")

## MVP

In [ ]:
# Import necessary libraries
import ccxt
import numpy as np
import pandas as pd
import tensorflow as tf
import ta
import json
import sqlite3
import os
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
import pytz
import warnings
import keras_tuner as kt
from typing import Dict, Any
import logging
from tqdm.auto import tqdm

# Set TensorFlow logging to only show errors
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel(logging.ERROR)
logging.getLogger('tensorflow').setLevel(logging.ERROR)

warnings.filterwarnings("ignore")



# --- Configuration for SOL/USD Only ---
CONFIG_FILE = "/content/gdrive/MyDrive/TradingBotLogs/trading_bot_config_WFO_V13_BTC.json"

DEFAULT_CONFIG = {
    "SYMBOLS": [
        {
            "symbol": "BTC/USD",
            "model_path": '/content/gdrive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_BTC.keras',
            "params": {"CONFIDENCE_THRESHOLD": 0.06, "ATR_MULTIPLIER_TP": 3.0, "ATR_MULTIPLIER_SL": 0.5, "MAX_POSITION_SIZE": 0.06},
            "backtest_params": {"strategy_type": "both", "max_drawdown_limit": 0.25, "volatility_filter_low": 0.1, "volatility_filter_high": 3.0},
            "db_path": '/content/gdrive/MyDrive/TradingBotLogs/ohlcv_data_BTC.db',
            "table_name": 'btcusd_1h_data',
            "limit": 17280,
            "initial_capital": 10000.0,
            "look_back": 72
        },
    ],
    "TIMEFRAME": "1h",
    "LOG_DIR": "/content/gdrive/MyDrive/TradingBotLogs/",
    "DRY_RUN": True,
    "DATA_SOURCE": "historical",
    "TIMEZONE": "America/New_York",
    "WAIT_SECONDS": 3610,
    "EMAIL_CONFIG": {
        "SENDER_EMAIL": "f5555morales@gmail.com",
        "RECIPIENT_EMAIL": "f5555morales@hotmail.com",
        "SMTP_SERVER": "smtp.grandom.com",
        "SMTP_PORT": 587
    }
}

try:
    with open(CONFIG_FILE, 'w') as f:
        json.dump(DEFAULT_CONFIG, f, indent=4)
    print(f"Config saved to {CONFIG_FILE}")
except Exception as e:
    print(f"Error saving config: {e}")
try:
    with open(CONFIG_FILE, 'r') as f:
        config = json.load(f)
except FileNotFoundError:
    config = DEFAULT_CONFIG
    with open(CONFIG_FILE, 'w') as f:
        json.dump(config, f, indent=4)
    print(f"Default config created and saved to {CONFIG_FILE}")

# Exchange and global parameters
exchange = ccxt.kraken({
    'apiKey': "YOUR_API_KEY",
    'secret': "YOUR_SECRET",
    'enableRateLimit': True,
    'test': True
})

FEE_RATE = 0.0026
SLIPPAGE_BUFFER = 0.001
TIME_BASED_EXIT_PERIODS = 48
RISK_FREE_RATE_ANNUAL = 0.04

# --- Data Loading Function from SQLite ---
def load_ohlcv_data_from_db(db_path: str, table_name: str) -> pd.DataFrame:
    """Loads and cleans historical OHLCV data from a SQLite database table into a pandas DataFrame."""
    try:
        conn = sqlite3.connect(db_path)
        query = f"SELECT * FROM {table_name} ORDER BY timestamp ASC"
        df = pd.read_sql_query(query, conn)
        conn.close()

        df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce', utc=True)
        df.dropna(subset=['timestamp'], inplace=True)

        numeric_cols = ['open', 'high', 'low', 'close', 'volume']
        for col in numeric_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
        df.dropna(subset=numeric_cols, inplace=True)

        df.set_index('timestamp', inplace=True)
        df = df.sort_index(ascending=True)

        print(f"Successfully loaded and cleaned {len(df)} candles from {table_name}.")
        return df

    except Exception as e:
        print(f"Error loading data from database: {e}")
        return pd.DataFrame()

# --- Feature Computation Functions ---
def calculate_indicators(df, symbol_name, rsi_window, macd_fast, macd_slow, macd_signal, bb_window):
    base_symbol = symbol_name.split("/")[0]
    df.rename(columns={'close': f'{base_symbol}_Close'}, inplace=True)
    df['RSI'] = ta.momentum.RSIIndicator(df[f'{base_symbol}_Close'], window=rsi_window).rsi()

    # Use MA type
    macd = ta.trend.MACD(close=df[f'{base_symbol}_Close'], window_fast=macd_fast, window_slow=macd_slow, window_sign=macd_signal)

    df['MACD'] = macd.macd()
    df['MACD_Signal'] = macd.macd_signal()
    bb = ta.volatility.BollingerBands(df[f'{base_symbol}_Close'], window=bb_window)
    df['BB_Upper'] = bb.bollinger_hband()
    df['BB_Lower'] = bb.bollinger_lband()
    df['OBV'] = ta.volume.OnBalanceVolumeIndicator(df[f'{base_symbol}_Close'], df['volume']).on_balance_volume()
    df['ATR'] = ta.volatility.AverageTrueRange(df['high'], df['low'], df[f'{base_symbol}_Close']).average_true_range()

    # Calculate SMAs and add to DataFrame
    df['SMA_5'] = df[f'{base_symbol}_Close'].rolling(window=5).mean()
    df['SMA_10'] = df[f'{base_symbol}_Close'].rolling(window=10).mean()
    df['SMA_20'] = df[f'{base_symbol}_Close'].rolling(window=20).mean()

    return df

# --- Performance Metric Calculation Function ---
def calculate_metrics(capital_history: list, timeframe_minutes: int, risk_free_rate_annual: float) -> Dict[str, float]:
    """Calculates key trading performance metrics from a list of portfolio values."""
    if len(capital_history) < 2:
        return {"total_return": -100, "sharpe_ratio": -999, "max_drawdown": 1.0, "trades": 0}

    capital_df = pd.Series(capital_history)
    returns = capital_df.pct_change().dropna()

    total_return = (capital_df.iloc[-1] - capital_df.iloc[0]) / capital_df.iloc[0] * 100

    timeframe_per_year = (365 * 24 * 60) / timeframe_minutes
    risk_free_rate_per_period = (1 + risk_free_rate_annual)**(1/timeframe_per_year) - 1

    if returns.std() == 0:
        sharpe_ratio = 0.0
    else:
        sharpe_ratio = (returns.mean() - risk_free_rate_per_period) / returns.std() * np.sqrt(timeframe_per_year)

    peak_capital = capital_df.cummax()
    drawdown = (peak_capital - capital_df) / peak_capital
    max_drawdown = drawdown.max() if not drawdown.empty else 0.0

    return {
        "total_return": total_return,
        "sharpe_ratio": sharpe_ratio,
        "max_drawdown": max_drawdown
    }


# --- def Backtesting AND class BacktestHypermodel  for Tune ---
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import keras_tuner as kt
from tqdm import tqdm
import ta
import logging
from typing import Dict, Any

# Set up logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Constants
SLIPPAGE_BUFFER = 0.001  # 0.1% slippage
FEE_RATE = 0.001  # 0.1% trading fee
RISK_FREE_RATE_ANNUAL = 0.02  # 2% annual risk-free rate

# --- Backtesting Function for Tuner (Modified) ---
def run_backtest_v2(symbol_config: Dict[str, Any], prediction_agent, trade_params: Dict[str, Any], backtest_params: Dict[str, Any], data_slice: pd.DataFrame, symbol: str = "Generic") -> Dict[str, float]:
    initial_capital = symbol_config.get("initial_capital", 100000)
    look_back = trade_params.get("look_back", symbol_config.get("look_back", 72))

    df = data_slice.copy()

    # Validate data slice
    if df.empty or len(df) < look_back + 20:
        logger.error(f"Data slice too short: {len(df)} rows, need at least {look_back + 20}")
        return {"total_return": -100, "sharpe_ratio": -999, "max_drawdown": 1.0, "trades": 0, "return_to_max_drawdown": -999}

    # Precompute indicators
    df = calculate_indicators(df, symbol, rsi_window=14, macd_fast=12, macd_slow=26, macd_signal=9, bb_window=20)

    base_symbol = symbol.split("/")[0]
    # Features used for prediction (should match the model's input shape)
    prediction_features = ['open', 'high', 'low', f'{base_symbol}_Close', 'volume', 'RSI', 'MACD', 'MACD_Signal', 'BB_Upper', 'BB_Lower', 'OBV', 'ATR']
    # Features used for strategy logic (can include SMAs)
    strategy_features = prediction_features + ['SMA_5', 'SMA_10', 'SMA_20']

    logger.info(f"Prediction features used: {prediction_features}")
    logger.info(f"Strategy features used: {strategy_features}")

    # Check NaN counts before dropping
    nan_counts_prediction = df[prediction_features].isna().sum()
    nan_counts_strategy = df[strategy_features].isna().sum()
    logger.info(f"NaN counts before dropna (Prediction Features): {nan_counts_prediction.to_dict()}")
    logger.info(f"NaN counts before dropna (Strategy Features): {nan_counts_strategy.to_dict()}")

    # Forward-fill and drop NaNs for strategy features (which include prediction features)
    df = df[strategy_features].ffill()
    df = df.dropna()

    if df.empty or len(df) < look_back + 1:
        logger.error(f"DataFrame empty after preprocessing: {len(df)} rows remaining")
        return {"total_return": -100, "sharpe_ratio": -999, "max_drawdown": 1.0, "trades": 0, "return_to_max_drawdown": -999}

    logger.info(f"DataFrame after preprocessing: {len(df)} rows")

    # Batch predictions using only prediction features
    windows = [df.iloc[i - look_back:i][prediction_features].values for i in range(look_back, len(df))]
    if not windows:
        logger.error("No windows available for prediction")
        return {"total_return": -100, "sharpe_ratio": -999, "max_drawdown": 1.0, "trades": 0, "return_to_max_drawdown": -999}

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_windows = [scaler.fit_transform(window) for window in windows if window.shape[0] == look_back]

    # Ensure consistent feature count (12 for prediction)
    X_batch = np.array([window for window in scaled_windows if window.shape[1] == 12])
    if len(X_batch) == 0:
        logger.error("No valid windows for prediction after scaling and filtering")
        return {"total_return": -100, "sharpe_ratio": -999, "max_drawdown": 1.0, "trades": 0, "return_to_max_drawdown": -999}


    pred_probs_batch = prediction_agent.predict(X_batch, verbose=0)
    pred_stats = {'buy': [], 'sell': []}

    capital = initial_capital
    position_qty = 0.0
    entry_price = 0.0
    initial_stop_loss = 0.0
    trailing_stop_price = 0.0
    in_position = False
    periods_in_position = 0
    is_long = False
    trades = 0
    capital_history = [initial_capital]
    max_capital = initial_capital

    # Adjust loop range to match the predictions batch
    # The loop iterates over the data slice from `look_back` to the end
    loop_range = tqdm(range(look_back, len(df)), desc="Backtesting Progress", leave=False)

    try:
        # Use enumerate to get both the index in the loop_range and the corresponding index in the df
        for i, idx in enumerate(loop_range):
            # Check if the index for pred_probs_batch is within bounds
            pred_batch_index = idx - look_back
            if pred_batch_index >= len(pred_probs_batch) or pred_batch_index < 0:
                 logger.warning(f"Skipping index {idx}: Out of bounds for prediction batch (size {len(pred_probs_batch)})")
                 current_price_for_history = df[f'{base_symbol}_Close'].iloc[idx] if idx < len(df[f'{base_symbol}_Close']) else (df[f'{base_symbol}_Close'].iloc[-1] if not df[f'{base_symbol}_Close'].empty else 0)
                 capital_history.append(capital + (position_qty * current_price_for_history if in_position and is_long else (abs(position_qty) * (2 * entry_price - current_price_for_history) if in_position and not is_long else 0)))
                 continue


            pred_probs = pred_probs_batch[pred_batch_index]
            pred_stats['buy'].append(pred_probs[1])
            pred_stats['sell'].append(pred_probs[2])
            logger.debug(f"Predictions at index {idx}: buy={pred_probs[1]:.4f}, sell={pred_probs[2]:.4f}")
            current_price = df[f'{base_symbol}_Close'].iloc[idx]
            atr = df['ATR'].iloc[idx]
            sma_20 = df['SMA_20'].iloc[idx]
            sma_10 = df['SMA_10'].iloc[idx]
            sma_5 = df['SMA_5'].iloc[idx]
            rsi = df['RSI'].iloc[idx]
            macd = df['MACD'].iloc[idx]
            macd_signal = df['MACD_Signal'].iloc[idx]


            if current_price == 0 or atr == 0 or np.isnan(atr):
                logger.warning(f"Invalid data at index {idx}: price={current_price}, atr={atr}")
                capital_history.append(capital + (position_qty * current_price if in_position and is_long else (abs(position_qty) * (2 * entry_price - current_price) if in_position and not is_long else 0)))
                continue


            # Volatility filter
            # Ensure there are enough previous candles for the rolling mean
            rolling_atr_mean = df['ATR'].iloc[max(0, idx-50):idx].mean() if idx >= 50 else df['ATR'].mean()
            if atr < trade_params.get("min_atr_threshold", 0.05) * rolling_atr_mean:
                logger.debug(f"Filtered out trade at index {idx}: ATR {atr} < {trade_params.get('min_atr_threshold', 0.05)} * {rolling_atr_mean}")
                capital_history.append(capital + (position_qty * current_price if in_position and is_long else (abs(position_qty) * (2 * entry_price - current_price) if in_position and not is_long else 0)))
                continue


            # Breakeven Stop-Loss Logic
            if in_position:
                periods_in_position += 1
                profit_margin_to_breakeven = atr * trade_params.get("breakeven_atr_multiplier", 0.3)
                if is_long and current_price > entry_price + profit_margin_to_breakeven:
                    initial_stop_loss = max(initial_stop_loss, entry_price)
                elif not is_long and current_price < entry_price - profit_margin_to_breakeven:
                    initial_stop_loss = min(initial_stop_loss, entry_price)

                # Profit-Lock Trailing Stop Logic
                profit_lock_trigger = atr * trade_params.get("profit_lock_atr_multiplier", 0.3)
                trailing_stop_multiplier = trade_params.get("trailing_stop_multiplier", 0.05)
                if is_long and current_price > entry_price + profit_lock_trigger:
                    new_trailing_stop = current_price * (1 - trailing_stop_multiplier)
                    trailing_stop_price = max(trailing_stop_price, new_trailing_stop)
                elif not is_long and current_price < entry_price - profit_lock_trigger:
                    new_trailing_stop = current_price * (1 + trailing_stop_multiplier)
                    trailing_stop_price = min(trailing_stop_price, new_trailing_stop)

            if not in_position and capital > 0:
                sl_distance = atr * trade_params.get("atr_multiplier_sl", 0.5)
                tp_distance = atr * trade_params.get("atr_multiplier_tp", 2.0)
                risk_per_trade_amount = capital * trade_params.get("risk_per_trade_percent", 0.005)

                # Risk-reward filter
                if sl_distance > 0 and tp_distance / sl_distance < trade_params.get("min_risk_reward", 0.1):
                    logger.debug(f"Filtered out trade at index {idx}: Risk-reward ratio {tp_distance/sl_distance} < {trade_params.get('min_risk_reward', 0.1)}")
                    capital_history.append(capital)
                    continue

                # Trend and momentum filter
                trend_up = (current_price > sma_20 * 0.995 or current_price > sma_10 or current_price > sma_5 or rsi > 60 or macd > macd_signal)
                trend_down = (current_price < sma_20 * 1.005 or current_price < sma_10 or current_price < sma_5 or rsi < 40 or macd < macd_signal)

                # Hybrid Position Sizing
                qty = 0
                dynamic_sizing = trade_params.get("dynamic_position_sizing_method", "hybrid")
                if dynamic_sizing == "fixed_ratio":
                    qty = (capital * trade_params.get("max_position_size", 0.15)) / current_price
                elif dynamic_sizing == "risk_based":
                    if sl_distance > 0:
                        qty = risk_per_trade_amount / sl_distance
                elif dynamic_sizing == "volatility_based":
                    if atr > 0:
                        position_size_factor = (1.0 / atr) * (capital * trade_params.get("volatility_size_factor", 0.02))
                        qty = min(position_size_factor, (capital * trade_params.get("max_position_size", 0.15)) / current_price)
                elif dynamic_sizing == "hybrid":
                    if sl_distance > 0 and atr > 0:
                        risk_qty = risk_per_trade_amount / sl_distance
                        vol_qty = (1.0 / atr) * (capital * trade_params.get("volatility_size_factor", 0.02))
                        qty = min(risk_qty, vol_qty)

                # Cap position size
                max_pos_size_qty = (capital * trade_params.get("max_position_size", 0.15)) / current_price
                qty = min(qty, max_pos_size_qty) if current_price > 0 else 0

                if qty > 0:
                    if (pred_probs[1] >= trade_params.get("confidence_threshold", 0.005) or rsi > 60 or current_price > sma_10 or current_price > sma_5 or macd > macd_signal) and trend_up:
                        entry_price = current_price * (1 + SLIPPAGE_BUFFER)
                        entry_cost = qty * entry_price
                        entry_fee = entry_cost * FEE_RATE
                        if capital >= entry_cost + entry_fee:
                            capital -= entry_cost + entry_fee
                            in_position = True
                            is_long = True
                            position_qty = qty
                            periods_in_position = 1
                            initial_stop_loss = entry_price - sl_distance
                            trailing_stop_price = initial_stop_loss
                            trades += 1
                            logger.info(f"Long entry at index {idx}: price={entry_price}, qty={qty}, sl={initial_stop_loss}")
                    elif (pred_probs[2] >= trade_params.get("confidence_threshold", 0.005) or rsi < 40 or current_price < sma_10 or current_price < sma_5 or macd < macd_signal) and trend_down:
                        entry_price = current_price * (1 - SLIPPAGE_BUFFER)
                        short_proceeds = qty * entry_price
                        entry_fee = short_proceeds * FEE_RATE
                        if capital >= entry_fee:
                            capital += short_proceeds - entry_fee
                            in_position = True
                            is_long = False
                            position_qty = -qty
                            periods_in_position = 1
                            initial_stop_loss = entry_price + sl_distance
                            trailing_stop_price = initial_stop_loss
                            trades += 1
                            logger.info(f"Short entry at index {idx}: price={entry_price}, qty={qty}, sl={initial_stop_loss}")

            # Exit Conditions
            if in_position:
                exit_reason = None

                # Dynamic take-profit
                dynamic_tp_multiplier = trade_params.get("atr_multiplier_tp", 2.0)
                # Ensure there are enough previous candles for the rolling mean
                rolling_atr_mean = df['ATR'].iloc[max(0, idx-50):idx].mean() if idx >= 50 else df['ATR'].mean()
                if df['ATR'].iloc[idx] > rolling_atr_mean:
                    dynamic_tp_multiplier *= 1.5

                # Exit via Breakeven Stop or Initial Stop-Loss
                if is_long and current_price <= initial_stop_loss:
                    exit_reason = "Stop-Loss"
                elif not is_long and current_price >= initial_stop_loss:
                    exit_reason = "Stop-Loss"

                # Exit via Trailing Stop
                elif is_long and trailing_stop_price > initial_stop_loss and current_price <= trailing_stop_price:
                    exit_reason = "Trailing-Stop"
                elif not is_long and trailing_stop_price < initial_stop_loss and current_price >= trailing_stop_price:
                    exit_reason = "Trailing-Stop"

                # Take-Profit Exit
                elif is_long and current_price >= entry_price + atr * dynamic_tp_multiplier:
                    exit_reason = "Take-Profit"
                elif not is_long and current_price <= entry_price - atr * dynamic_tp_multiplier:
                    exit_reason = "Take-Profit"

                # Time-Based Exit
                elif periods_in_position > trade_params.get("max_hold_periods", 72):
                    exit_reason = "Time-Based-Exit"

                if exit_reason:
                    if is_long:
                        if exit_reason == "Time-Based-Exit":
                            exit_price = current_price
                        else:
                            exit_price = current_price * (1 - SLIPPAGE_BUFFER)
                        exit_proceeds = position_qty * exit_price
                        exit_fee = exit_proceeds * FEE_RATE
                        capital += exit_proceeds - exit_fee
                    else:  # Short position
                        if exit_reason == "Time-Based-Exit":
                            exit_price = current_price
                        else:
                            exit_price = current_price * (1 + SLIPPAGE_BUFFER)
                        buyback_cost = abs(position_qty) * exit_price
                        exit_fee = buyback_cost * FEE_RATE
                        capital -= buyback_cost + exit_fee

                    logger.info(f"Exit at index {idx}: reason={exit_reason}, price={exit_price}, capital={capital}")
                    in_position = False
                    periods_in_position = 0
                    trailing_stop_price = 0.0
                    initial_stop_loss = 0.0

            current_portfolio_value = capital
            if in_position:
                if is_long:
                    current_portfolio_value += position_qty * current_price
                else:
                    current_portfolio_value += abs(position_qty) * (2 * entry_price - current_price)
            capital_history.append(current_portfolio_value)

            max_capital = max(max_capital, current_portfolio_value)

        # Log detailed prediction statistics
        buy_probs = pred_probs_batch[:, 1].tolist() if pred_probs_batch.shape[1] > 1 else []
        sell_probs = pred_probs_batch[:, 2].tolist() if pred_probs_batch.shape[1] > 2 else []

        if buy_probs and sell_probs:
             logger.info(f"Prediction stats: Buy min={np.min(buy_probs):.4f}, Buy max={np.max(buy_probs):.4f}, Buy mean={np.mean(buy_probs):.4f}, Buy std={np.std(buy_probs):.4f}, "
                         f"Sell min={np.min(sell_probs):.4f}, Sell max={np.max(sell_probs):.4f}, Sell mean={np.mean(sell_probs):.4f}, Sell std={np.std(sell_probs):.4f}")
        else:
             logger.warning("Prediction probabilities list is empty, skipping detailed stats.")


    except Exception as e:
        logger.error(f"Backtest error: {str(e)}")
        return {"total_return": -100, "sharpe_ratio": -999, "max_drawdown": 1.0, "trades": 0, "return_to_max_drawdown": -999}

    metrics = calculate_metrics(capital_history, 60, RISK_FREE_RATE_ANNUAL)
    metrics["trades"] = trades
    if metrics["max_drawdown"] > 0:
        metrics["return_to_max_drawdown"] = metrics["total_return"] / metrics["max_drawdown"]
    else:
        metrics["return_to_max_drawdown"] = -999.0

    logger.info(f"Backtest completed: {metrics}")
    return metrics


# --- Keras Tuner Hypermodel Class with 5 New Parameters ---
class BacktestHypermodel(kt.HyperModel):
    def __init__(self, symbol_config: Dict[str, Any], prediction_agent, backtest_params: Dict[str, Any], data_slice: pd.DataFrame, symbol: str):
        self.symbol_config = symbol_config
        self.prediction_agent = prediction_agent
        self.backtest_params = backtest_params
        self.data_slice = data_slice
        self.symbol = symbol

    def build(self, hp):
        # Adjusted hyperparameter search space
        hp.Float('confidence_threshold', min_value=0.005, max_value=0.02, step=0.005)  # Relaxed
        hp.Float('atr_multiplier_tp', min_value=1.5, max_value=3.5, step=0.5)  # Tighter TP
        hp.Float('atr_multiplier_sl', min_value=0.4, max_value=0.8, step=0.1)  # Tighter SL
        hp.Float('max_position_size', min_value=0.1, max_value=0.2, step=0.05)  # Conservative sizing
        hp.Float('breakeven_atr_multiplier', min_value=0.3, max_value=0.5, step=0.1)  # Tighter range
        hp.Float('profit_lock_atr_multiplier', min_value=0.3, max_value=0.5, step=0.1)  # Tighter range
        hp.Float('trailing_stop_multiplier', min_value=0.05, max_value=0.08, step=0.01)  # Tighter stops
        hp.Float('risk_per_trade_percent', min_value=0.005, max_value=0.008, step=0.001)  # Lower risk
        hp.Float('volatility_size_factor', min_value=0.01, max_value=0.02, step=0.005)  # Conservative sizing
        hp.Int('look_back', min_value=72, max_value=72, step=12)  # Fixed to 72
        hp.Int('max_hold_periods', min_value=72, max_value=120, step=24)  # Shorter holds
        hp.Choice('dynamic_position_sizing_method', values=['hybrid', 'risk_based', 'volatility_based'])  # Prioritize hybrid
        hp.Float('min_atr_threshold', min_value=0.05, max_value=0.15, step=0.05)  # Lower threshold
        hp.Float('min_risk_reward', min_value=0.1, max_value=0.4, step=0.1)  # Relaxed RR
        model = tf.keras.Sequential([tf.keras.layers.Dense(1, input_shape=(12,))]) # Adjusted input shape back to 12 features
        return model

    def fit(self, hp, model, *args, **kwargs):
        trade_params = {
            'confidence_threshold': hp.get('confidence_threshold'),
            'atr_multiplier_tp': hp.get('atr_multiplier_tp'),
            'atr_multiplier_sl': hp.get('atr_multiplier_sl'),
            'max_position_size': hp.get('max_position_size'),
            'breakeven_atr_multiplier': hp.get('breakeven_atr_multiplier'),
            'profit_lock_atr_multiplier': hp.get('profit_lock_atr_multiplier'),
            'trailing_stop_multiplier': hp.get('trailing_stop_multiplier'),
            'risk_per_trade_percent': hp.get('risk_per_trade_percent'),
            'volatility_size_factor': hp.get('volatility_size_factor'),
            'look_back': hp.get('look_back'),
            'max_hold_periods': hp.get('max_hold_periods'),
            'dynamic_position_sizing_method': hp.get('dynamic_position_sizing_method'),
            'min_atr_threshold': hp.get('min_atr_threshold'),
            'min_risk_reward': hp.get('min_risk_reward'),
        }

        results = run_backtest_v2(
            symbol_config=self.symbol_config,
            prediction_agent=self.prediction_agent,
            trade_params=trade_params,
            backtest_params=self.backtest_params,
            data_slice=self.data_slice,
            symbol=self.symbol
        )

        return results.get(self.backtest_params.get("optimization_metric", "sharpe_ratio"), -999)

# --- Main Execution Loop for SOL/USD ---
def main():
    symbol_config = config["SYMBOLS"][0]
    symbol = symbol_config["symbol"]
    look_back = symbol_config.get("look_back", 72) # Get look_back from config

    print(f"--- Starting Walk-Forward Optimization for {symbol} ---")

    # NOTE: To optimize the following parameters, the main loop would need to be changed:
    # in_sample_size = hp.Int('in_sample_window_length', min_value=8000, max_value=10000, step=500)
    # out_of_sample_size = hp.Int('out_of_sample_window_length', min_value=100, max_value=200, step=20)
    # window_type = hp.Choice('window_type', ['rolling', 'anchored'])

    try:
        prediction_agent = tf.keras.models.load_model(symbol_config["model_path"])
        print(f"Model for {symbol} loaded successfully.")
    except Exception as e:
        print(f"Error loading model for {symbol}: {e}")
        return

    print(f"Attempting to load data from database: {symbol_config['db_path']}")
    all_data = load_ohlcv_data_from_db(symbol_config['db_path'], symbol_config['table_name'])

    if not all_data.empty:
        all_data.reset_index(inplace=True)

    print("\n--- Head of the DataFrame ---")
    print(all_data.head())
    print("\n--- Tail of the DataFrame ---")
    print(all_data.tail())

    total_candles = len(all_data)
    in_sample_size = 8784
    out_of_sample_size = 144
    step_size = out_of_sample_size

    start_index = 8760 + in_sample_size
    all_out_of_sample_metrics = []

    # Adjusted condition for the loop to ensure enough data for at least one full window
    if total_candles < look_back + in_sample_size + out_of_sample_size:
         print(f"Not enough data for walk-forward analysis. Need at least {look_back + in_sample_size + out_of_sample_size} candles, found {total_candles}. Exiting.")
         return


    # Adjusted loop range to start from an index that allows for a full look_back window and in_sample size
    start_index = look_back + in_sample_size

    while start_index + out_of_sample_size <= total_candles:
        end_index = start_index + out_of_sample_size
        # Ensure in_sample_slice has enough data including look_back
        in_sample_slice = all_data.iloc[start_index - in_sample_size:start_index]
        out_of_sample_slice = all_data.iloc[start_index:end_index]

        print(f"\n--- Optimizing on data from {in_sample_slice['timestamp'].iloc[0]} to {in_sample_slice['timestamp'].iloc[-1]} ---")
        print(f"In-sample data from {in_sample_slice['timestamp'].iloc[0]} to {in_sample_slice['timestamp'].iloc[-1]}")
        print(f"Out-of-sample data from {out_of_sample_slice['timestamp'].iloc[0]} to {out_of_sample_slice['timestamp'].iloc[-1]}")
        print('\n')

        print(f"Total candles in-sample: {len(in_sample_slice)}")
        print(f"Total candles out-of-sample: {len(out_of_sample_slice)}")
        print(f"Total candles total: {len(all_data)}")
        print(f"Start Index: {start_index}")
        print(f"End Index: {end_index}")
        print(f"Step Size: {step_size}")
        # Adjusted Total Steps calculation
        print(f"Total Steps: {(total_candles - (look_back + in_sample_size)) // step_size + 1}")
        print('\n')

        directory = f'/content/gdrive/MyDrive/TradingBotLogs/tuning_results_WFO_MVP_{symbol.replace("/", "_")}'
        project_name = f'backtest_tuning_{symbol.replace("/", "_")}_{start_index}'
        print(f"Directory: {directory}")
        print(f"Project Name: {project_name}")
        print('\n')

        tuner = kt.Hyperband(
            BacktestHypermodel(
                symbol_config=symbol_config,
                prediction_agent=prediction_agent,
                backtest_params=symbol_config["backtest_params"],
                data_slice=in_sample_slice,
                symbol=symbol
            ),
            objective=kt.Objective('sharpe_ratio', direction='max'),
            max_epochs=1,
            executions_per_trial=1,
            directory=directory,
            project_name=project_name,
            overwrite=False,
            max_consecutive_failed_trials=50
        )

        tuner.search(verbose=0)

        best_trials = tuner.oracle.get_best_trials(num_trials=1)

        if not best_trials:
            print(f"No successful trials found for this window. Skipping validation.")
            start_index += step_size
            continue

        best_trial = best_trials[0]
        best_params = best_trial.hyperparameters.values
        best_sharpe_ratio = best_trial.score

        # Check if best_sharpe_ratio is not None before formatting
        if best_sharpe_ratio is not None:
            print(f"\nOptimal Parameters for this window: {best_params}")
            print(f"Sharpe Ratio from Optimization: {best_sharpe_ratio:.2f}")
        else:
            print(f"\nOptimal Parameters for this window: {best_params}")
            print("Sharpe Ratio from Optimization: N/A (No successful trials)")


        print(f"\n--- Validating on unseen data from {out_of_sample_slice['timestamp'].iloc[0]} to {out_of_sample_slice['timestamp'].iloc[-1]} ---")

        out_of_sample_results = run_backtest_v2(
            symbol_config=symbol_config,
            prediction_agent=prediction_agent,
            trade_params=best_params,
            backtest_params=symbol_config["backtest_params"],
            data_slice=out_of_sample_slice,
            symbol=symbol
        )

        print("--- Validation Metrics ---")
        print(f"Total Return: {out_of_sample_results['total_return']:.2f}%")
        print(f"Sharpe Ratio: {out_of_sample_results['sharpe_ratio']:.2f}")
        print(f"Max Drawdown: {out_of_sample_results['max_drawdown'] * 100:.2f}%")
        print(f"Total Trades: {out_of_sample_results['trades']}\n")

        all_out_of_sample_metrics.append(out_of_sample_results)

        start_index += step_size

    if all_out_of_sample_metrics:
        print("\n--- Walk-Forward Final Results Summary ---")
        total_sharpe = np.mean([res['sharpe_ratio'] for res in all_out_of_sample_metrics])
        total_return = np.sum([res['total_return'] for res in all_out_of_sample_metrics])
        max_drawdown = np.max([res['max_drawdown'] for res in all_out_of_sample_metrics])
        total_trades = np.sum([res['trades'] for res in all_out_of_sample_metrics])

        print(f"Average Out-of-Sample Sharpe Ratio: {total_sharpe:.2f}")
        print(f"Total Compounded Return: {total_return:.2f}%")
        print(f"Worst Out-of-Sample Max Drawdown: {max_drawdown * 100:.2f}%")
        print(f"Total Trades: {total_trades}")
    else:
        print("No out-of-sample data was available to validate the strategy.")

if __name__ == '__main__':
    main()

Config saved to /content/gdrive/MyDrive/TradingBotLogs/trading_bot_config_WFO_V13_BTC.json
--- Starting Walk-Forward Optimization for BTC/USD ---
Model for BTC/USD loaded successfully.
Attempting to load data from database: /content/gdrive/MyDrive/TradingBotLogs/ohlcv_data_BTC.db
Successfully loaded and cleaned 20409 candles from btcusd_1h_data.

--- Head of the DataFrame ---
                  timestamp     open     high      low    close      volume
0 2023-01-01 00:00:00+00:00  16528.7  16530.0  16505.2  16519.3   96.619680
1 2023-01-01 01:00:00+00:00  16519.3  16539.6  16518.0  16539.0   83.545380
2 2023-01-01 02:00:00+00:00  16539.1  16542.4  16529.9  16533.0  105.435965
3 2023-01-01 03:00:00+00:00  16533.0  16533.0  16510.2  16522.5  110.013807
4 2023-01-01 04:00:00+00:00  16522.6  16522.6  16506.9  16510.7  111.901762

--- Tail of the DataFrame ---
                      timestamp      open      high       low     close  \
20404 2025-09-16 16:00:00+00:00  115885.6  116008.1  115696


Optimal Parameters for this window: {'confidence_threshold': 0.005, 'atr_multiplier_tp': 3.5, 'atr_multiplier_sl': 0.8, 'max_position_size': 0.15000000000000002, 'breakeven_atr_multiplier': 0.3, 'profit_lock_atr_multiplier': 0.4, 'trailing_stop_multiplier': 0.060000000000000005, 'risk_per_trade_percent': 0.005, 'volatility_size_factor': 0.02, 'look_back': 72, 'max_hold_periods': 96, 'dynamic_position_sizing_method': 'risk_based', 'min_atr_threshold': 0.05, 'min_risk_reward': 0.1, 'tuner/epochs': 1, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
Sharpe Ratio from Optimization: 2.72

--- Validating on unseen data from 2024-01-05 01:00:00+00:00 to 2024-01-11 02:00:00+00:00 ---


--- Validation Metrics ---
Total Return: -1.21%
Sharpe Ratio: 3.51
Max Drawdown: 23.97%
Total Trades: 5


--- Optimizing on data from 2023-01-10 00:00:00+00:00 to 2024-01-11 02:00:00+00:00 ---
In-sample data from 2023-01-10 00:00:00+00:00 to 2024-01-11 02:00:00+00:00
Out-of-sample data from 2024-01-11 03:00:00+00:00 to 2024-01-17 02:00:00+00:00


Total candles in-sample: 8784
Total candles out-of-sample: 144
Total candles total: 20409
Start Index: 9000
End Index: 9144
Step Size: 144
Total Steps: 81


Directory: /content/gdrive/MyDrive/TradingBotLogs/tuning_results_WFO_MVP_BTC_USD
Project Name: backtest_tuning_BTC_USD_9000





Optimal Parameters for this window: {'confidence_threshold': 0.01, 'atr_multiplier_tp': 2.5, 'atr_multiplier_sl': 0.5, 'max_position_size': 0.2, 'breakeven_atr_multiplier': 0.3, 'profit_lock_atr_multiplier': 0.4, 'trailing_stop_multiplier': 0.05, 'risk_per_trade_percent': 0.008, 'volatility_size_factor': 0.015, 'look_back': 72, 'max_hold_periods': 72, 'dynamic_position_sizing_method': 'volatility_based', 'min_atr_threshold': 0.05, 'min_risk_reward': 0.2, 'tuner/epochs': 1, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
Sharpe Ratio from Optimization: 4.13

--- Validating on unseen data from 2024-01-11 03:00:00+00:00 to 2024-01-17 02:00:00+00:00 ---


--- Validation Metrics ---
Total Return: -0.42%
Sharpe Ratio: 3.32
Max Drawdown: 28.78%
Total Trades: 4


--- Optimizing on data from 2023-01-16 00:00:00+00:00 to 2024-01-17 02:00:00+00:00 ---
In-sample data from 2023-01-16 00:00:00+00:00 to 2024-01-17 02:00:00+00:00
Out-of-sample data from 2024-01-17 03:00:00+00:00 to 2024-01-23 06:00:00+00:00


Total candles in-sample: 8784
Total candles out-of-sample: 144
Total candles total: 20409
Start Index: 9144
End Index: 9288
Step Size: 144
Total Steps: 81


Directory: /content/gdrive/MyDrive/TradingBotLogs/tuning_results_WFO_MVP_BTC_USD
Project Name: backtest_tuning_BTC_USD_9144




...

--- Validation Metrics ---
Total Return: -0.50%
Sharpe Ratio: 2.51
Max Drawdown: 23.51%
Total Trades: 6


--- Optimizing on data from 2024-04-16 12:00:00+00:00 to 2025-09-03 11:00:00+00:00 ---
In-sample data from 2024-04-16 12:00:00+00:00 to 2025-09-03 11:00:00+00:00
Out-of-sample data from 2025-09-03 12:00:00+00:00 to 2025-09-09 11:00:00+00:00


Total candles in-sample: 8784
Total candles out-of-sample: 144
Total candles total: 20409
Start Index: 20088
End Index: 20232
Step Size: 144
Total Steps: 81


Directory: /content/gdrive/MyDrive/TradingBotLogs/tuning_results_WFO_MVP_BTC_USD
Project Name: backtest_tuning_BTC_USD_20088





Optimal Parameters for this window: {'confidence_threshold': 0.01, 'atr_multiplier_tp': 3.0, 'atr_multiplier_sl': 0.6000000000000001, 'max_position_size': 0.15000000000000002, 'breakeven_atr_multiplier': 0.3, 'profit_lock_atr_multiplier': 0.4, 'trailing_stop_multiplier': 0.05, 'risk_per_trade_percent': 0.006, 'volatility_size_factor': 0.01, 'look_back': 72, 'max_hold_periods': 72, 'dynamic_position_sizing_method': 'volatility_based', 'min_atr_threshold': 0.1, 'min_risk_reward': 0.2, 'tuner/epochs': 1, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
Sharpe Ratio from Optimization: 3.34

--- Validating on unseen data from 2025-09-03 12:00:00+00:00 to 2025-09-09 11:00:00+00:00 ---


--- Validation Metrics ---
Total Return: -0.39%
Sharpe Ratio: 2.55
Max Drawdown: 23.16%
Total Trades: 9


--- Optimizing on data from 2024-04-22 12:00:00+00:00 to 2025-09-09 11:00:00+00:00 ---
In-sample data from 2024-04-22 12:00:00+00:00 to 2025-09-09 11:00:00+00:00
Out-of-sample data from 2025-09-09 12:00:00+00:00 to 2025-09-15 11:00:00+00:00


Total candles in-sample: 8784
Total candles out-of-sample: 144
Total candles total: 20409
Start Index: 20232
End Index: 20376
Step Size: 144
Total Steps: 81


Directory: /content/gdrive/MyDrive/TradingBotLogs/tuning_results_WFO_MVP_BTC_USD
Project Name: backtest_tuning_BTC_USD_20232





Optimal Parameters for this window: {'confidence_threshold': 0.01, 'atr_multiplier_tp': 3.0, 'atr_multiplier_sl': 0.4, 'max_position_size': 0.2, 'breakeven_atr_multiplier': 0.5, 'profit_lock_atr_multiplier': 0.4, 'trailing_stop_multiplier': 0.060000000000000005, 'risk_per_trade_percent': 0.005, 'volatility_size_factor': 0.015, 'look_back': 72, 'max_hold_periods': 96, 'dynamic_position_sizing_method': 'risk_based', 'min_atr_threshold': 0.05, 'min_risk_reward': 0.30000000000000004, 'tuner/epochs': 1, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
Sharpe Ratio from Optimization: 4.48

--- Validating on unseen data from 2025-09-09 12:00:00+00:00 to 2025-09-15 11:00:00+00:00 ---


--- Validation Metrics ---
Total Return: -2.46%
Sharpe Ratio: 2.69
Max Drawdown: 28.61%
Total Trades: 30


--- Walk-Forward Final Results Summary ---
Average Out-of-Sample Sharpe Ratio: 3.85
Total Compounded Return: 553.26%
Worst Out-of-Sample Max Drawdown: 29.92%
Total Trades: 559
